We need to make sure the breed as given by the pound is converted to a breed from dogtime. Let's try using the Levenshtein distance.

In [18]:
import pandas as pd
import numpy as np
from Levenshtein import distance as LD

In [19]:
#User input desired scores
x0 = np.array([1,2,3,4,5,4,3,2,1,3,2])

In [20]:
breed_info = pd.read_csv("breed_info.csv")
breed_info = breed_info.drop(208)
breed_list = [l.lower() for l in breed_info["Dog Breed"]]
breed_info_trunc = breed_info[["Dog Breed",
           "Adapts Well to Apartment Living",
           "Good For Novice Owners",
           "Tolerates Being Alone",
           "Incredibly Kid Friendly Dogs",
           "Friendly Toward Strangers",
           "Amount Of Shedding",
           "Easy To Groom",
           "Size",
           "Intelligence",
           "Tendency To Bark Or Howl",
           "Exercise Needs"] ]
breed_info_trunc["Dog Breed"] = breed_info_trunc["Dog Breed"].map(lambda x: x.lower())




dog_inventory = pd.read_csv("found_list.csv", index_col=0)
dog_inventory = dog_inventory.reset_index()
dog_inventory = dog_inventory.drop(["index"],axis=1)
breed_inventory = dog_inventory["Breed"]

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [21]:
def breed_preprocess(s):
    breed = s.lower()
    if "pit bull" in breed:
        breed = "american pit bull terrirer"
    breed = breed.replace("germ sh ", "german shorthaired ")
    breed = {True: "west highland white terrier", False: breed}[breed == "west highland"]
    breed = {True: "poodle", False: breed}["poodle" in breed]
    breed_split = breed.split()
    for i in range(len(breed_split)):
        breed_split[i] = {True: "american", False: breed_split[i] }[breed_split[i] == "am" ]
        breed_split[i] = {True: "terrier", False: breed_split[i] }[breed_split[i] == "ter" ]
        breed_split[i] = {True: "terrier", False: breed_split[i] }[breed_split[i] == "terr" ]
        breed_split[i] = {True: "german", False: breed_split[i] }[breed_split[i] == "germ" ]
        breed_split[i] = {True: "miniature", False: breed_split[i] }[breed_split[i] == "min" ]
        breed_split[i] = {True: "saint", False: breed_split[i] }[breed_split[i] == "st" ]
        breed_split[i] = {True: "", False: breed_split[i] }[breed_split[i] == "mix" ]
        breed_split[i] = {True: "australian", False: breed_split[i] }[breed_split[i] == "aust" ]
        breed_split[i] = {True: "retriever", False: breed_split[i] }[breed_split[i] == "retr" ]
        
    if "miniature" in breed_split:
        breed_split.insert(0, breed_split.pop( breed_split.index("miniature") ))
    ret_string = breed_split[0]
    for j in range(1,len(breed_split)):
        ret_string = ret_string + " " + breed_split[j]
    return ret_string
    

In [22]:
def breed_fix(s):
    s_new = breed_preprocess(s)
    min_dist = 100
    current_breed = ""
    for breed in breed_list:
        if LD(s_new, breed) < min_dist:
            min_dist = LD(s_new, breed)
            current_breed = breed
            
    
    return current_breed

In [23]:
dog_inventory["Mapped_Breed"] = breed_inventory.map(breed_fix)

In [25]:
user_breed_dist = []
for dog in dog_inventory["Mapped_Breed"]:
    user_breed_dist.append((np.linalg.norm(breed_info_trunc.loc[breed_info_trunc["Dog Breed"] == dog ].as_matrix()[0][1:12] - x0, 3)))

In [26]:
dog_inventory["Recommendation Score"] =  user_breed_dist

In [28]:
dog_inventory.sort_values(by = "Recommendation Score", ascending=False).head(5)

,ID,Sex,Age,Weight,Color,Breed,Intake Jurisdiction,Intake Date,Mapped_Breed,Recommendation Score
60,A1570849,F,3 Years,12.0 LBS,GRAY TAN,POODLE MIN,YORBA LINDA,4/13/2018,poodle,6.009245
61,A1570850,M,2 Years,15.0 LBS,WHITE,POODLE MIN,SANTA ANA,4/13/2018,poodle,6.009245
62,A1568093,M,1 Year 1 Month,5.3 LBS,WHITE TAN,POODLE MIN MIX,TUSTIN,3/26/2018,poodle,6.009245
59,A1570429,F,3 Years,21.6 LBS,WHITE,POODLE MIN,SANTA ANA,4/10/2018,poodle,6.009245
21,A1570778,M,1 Year,14.0 LBS,BROWN BLACK,DACHSHUND MIX,FULLERTON,4/12/2018,dachshund,5.593445


In [8]:
#breed_matrix = breed_info_trunc.as_matrix()
#def breed_rec(x):
#    min_dist = 10000
#    recommend_breed = ''
#    for j in range(len(breed_matrix)):
#        current_breed = breed_matrix[j][0]
#        current_breed_data = breed_matrix[j][1:12]
#        if np.linalg.norm(x - current_breed_data) < min_dist:
#            min_dist = np.linalg.norm(x - current_breed_data)
#            recommend_breed = current_breed
#    return recommend_breed, min_dist
        
        

In [9]:
#breed_rec_dict = {}
#for j in range(len(breed_matrix)):
#    current_breed = breed_matrix[j][0]
#    current_breed_data = breed_matrix[j][1:12]
#    dist = np.linalg.norm(x0 - current_breed_data)
#    breed_rec_dict[current_breed] = dist